# Huomiomekanismit ja transformerit

Yksi toistoverkkojen suurimmista heikkouksista on, että kaikki sanat sekvenssissä vaikuttavat tulokseen samalla tavalla. Tämä johtaa heikkoon suorituskykyyn tavanomaisilla LSTM-enkooderi-dekooderi-malleilla sekvenssistä sekvenssiin -tehtävissä, kuten nimettyjen entiteettien tunnistuksessa ja konekäännöksessä. Todellisuudessa tietyillä syötteen sanoilla on usein suurempi vaikutus peräkkäisiin tulosteisiin kuin toisilla.

Ajatellaan sekvenssistä sekvenssiin -mallia, kuten konekäännöstä. Se toteutetaan kahdella toistoverkolla, joissa yksi verkko (**enkooderi**) tiivistää syötteen piilotilaan ja toinen verkko (**dekooderi**) purkaa tämän piilotilan käännetyksi tulokseksi. Tämän lähestymistavan ongelmana on, että verkon lopputilalla on vaikeuksia muistaa lauseen alkua, mikä heikentää mallin laatua pitkissä lauseissa.

**Huomiomekanismit** tarjoavat tavan painottaa kunkin syötevektorin kontekstuaalista vaikutusta RNN:n jokaisessa tulosennusteessa. Tämä toteutetaan luomalla oikopolkuja syötteen RNN:n välitilojen ja tulos-RNN:n välille. Näin ollen, kun tuotetaan tulossymbolia $y_t$, otamme huomioon kaikki syötteen piilotilat $h_i$, eri painokertoimilla $\alpha_{t,i}$. 

![Kuva, joka näyttää enkooderi/dekooderi-mallin additiivisella huomiokerroksella](../../../../../translated_images/fi/encoder-decoder-attention.7a726296894fb567.webp)
*Enkooderi-dekooderi-malli additiivisella huomiomekanismilla [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), lainattu [tästä blogikirjoituksesta](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Huomiomatriisi $\{\alpha_{i,j}\}$ edustaa sitä, kuinka paljon tietyt syötteen sanat vaikuttavat tietyn sanan muodostumiseen tulossekvenssissä. Alla on esimerkki tällaisesta matriisista:

![Kuva, joka näyttää esimerkkikohdistuksen RNNsearch-50:llä, otettu Bahdanau - arviz.org](../../../../../translated_images/fi/bahdanau-fig3.09ba2d37f202a6af.webp)

*Kuva otettu [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Kuva 3)*

Huomiomekanismit ovat vastuussa suuresta osasta nykyistä tai lähes nykyistä huipputasoa luonnollisen kielen käsittelyssä. Huomion lisääminen kuitenkin kasvattaa merkittävästi mallin parametrien määrää, mikä aiheutti skaalausongelmia RNN:ien kanssa. RNN:ien skaalaamisen keskeinen rajoite on, että mallien toistuva luonne tekee koulutuksen eräajosta ja rinnakkaistamisesta haastavaa. RNN:ssä jokainen sekvenssin elementti täytyy käsitellä järjestyksessä, mikä tarkoittaa, ettei sitä voida helposti rinnakkaistaa.

Huomiomekanismien käyttöönotto yhdessä tämän rajoitteen kanssa johti nykyisten huipputason Transformer-mallien luomiseen, joita käytämme tänään, kuten BERT ja OpenGPT3.

## Transformer-mallit

Sen sijaan, että jokaisen edellisen ennusteen konteksti välitettäisiin seuraavaan arviointivaiheeseen, **transformer-mallit** käyttävät **paikkakoodauksia** ja huomiota tallentaakseen annetun syötteen kontekstin annetussa tekstin ikkunassa. Alla oleva kuva näyttää, kuinka paikkakoodaukset ja huomio voivat tallentaa kontekstin annetussa ikkunassa.

![Animoitu GIF, joka näyttää, kuinka arvioinnit suoritetaan transformer-malleissa.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Koska jokainen syötteen sijainti kartoitetaan itsenäisesti jokaiseen tulosteen sijaintiin, transformerit voivat rinnakkaistaa paremmin kuin RNN:t, mikä mahdollistaa paljon suuremmat ja ilmaisukykyisemmät kielimallit. Jokainen huomiointipää voi oppia erilaisia suhteita sanojen välillä, mikä parantaa luonnollisen kielen käsittelyn tehtäviä.

**BERT** (Bidirectional Encoder Representations from Transformers) on erittäin suuri monikerroksinen transformer-verkko, jossa on 12 kerrosta *BERT-base*-mallissa ja 24 kerrosta *BERT-large*-mallissa. Malli esikoulutetaan ensin suurella tekstikorpuksella (Wikipedia + kirjat) käyttämällä valvomattua koulutusta (ennustamalla peitettyjä sanoja lauseessa). Esikoulutuksen aikana malli omaksuu merkittävän määrän kielen ymmärrystä, jota voidaan hyödyntää muilla aineistoilla hienosäädön avulla. Tätä prosessia kutsutaan **siirto-oppimiseksi**. 

![Kuva osoitteesta http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/fi/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Transformer-arkkitehtuureista on monia variaatioita, kuten BERT, DistilBERT, BigBird, OpenGPT3 ja muita, joita voidaan hienosäätää. [HuggingFace-paketti](https://github.com/huggingface/) tarjoaa kirjaston monien näiden arkkitehtuurien kouluttamiseen PyTorchilla. 

## BERT:n käyttäminen tekstiluokitteluun

Katsotaanpa, kuinka voimme käyttää esikoulutettua BERT-mallia perinteisen tehtävämme ratkaisemiseen: sekvenssiluokitteluun. Luokittelemme alkuperäisen AG News -aineistomme.

Ensiksi ladataan HuggingFace-kirjasto ja aineistomme:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Koska käytämme valmiiksi koulutettua BERT-mallia, meidän täytyy käyttää sille tarkoitettua tokenisoijaa. Ensiksi lataamme tokenisoijan, joka liittyy valmiiksi koulutettuun BERT-malliin.

HuggingFace-kirjasto sisältää valmiiksi koulutettujen mallien arkiston, jota voit käyttää yksinkertaisesti määrittämällä mallin nimen `from_pretrained`-funktioiden argumenttina. Kaikki mallin tarvitsemat binääritiedostot ladataan automaattisesti.

Joissain tilanteissa saatat kuitenkin joutua lataamaan omia mallejasi. Tällöin voit määrittää hakemiston, joka sisältää kaikki tarvittavat tiedostot, kuten tokenisoijan parametrit, `config.json`-tiedoston mallin parametreilla, binääripainot jne.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer`-objekti sisältää `encode`-funktion, jota voidaan käyttää suoraan tekstin koodaamiseen:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Sitten luodaan iteraattorit, joita käytämme koulutuksen aikana datan käsittelyyn. Koska BERT käyttää omaa koodausfunktiotaan, meidän täytyy määritellä täyttöfunktio, joka on samanlainen kuin aiemmin määrittelemämme `padify`:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

Tapauksessamme käytämme valmiiksi koulutettua BERT-mallia nimeltä `bert-base-uncased`. Ladataan malli käyttämällä `BertForSequenceClassification`-pakettia. Tämä varmistaa, että mallillamme on jo tarvittava luokitteluun tarkoitettu arkkitehtuuri, mukaan lukien lopullinen luokitin. Näet varoitusviestin, joka ilmoittaa, että lopullisen luokittimen painot eivät ole alustettuja, ja malli vaatisi esikoulutusta - tämä on täysin normaalia, sillä juuri sitä olemme tekemässä!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Nyt olemme valmiita aloittamaan koulutuksen! Koska BERT on jo valmiiksi esikoulutettu, haluamme aloittaa melko pienellä oppimisnopeudella, jotta emme tuhoa alkuperäisiä painoja.

Kaiken raskaan työn hoitaa `BertForSequenceClassification`-malli. Kun kutsumme mallia koulutusdatan kanssa, se palauttaa sekä häviön että verkon ulostulon syöteminierälle. Käytämme häviötä parametrien optimointiin (`loss.backward()` suorittaa taaksepäin kulkevan vaiheen) ja `out`-arvoa koulutustarkkuuden laskemiseen vertaamalla saatuja luokkia `labs` (laskettu käyttäen `argmax`) odotettuihin `labels`-arvoihin.

Prosessin hallitsemiseksi keräämme häviön ja tarkkuuden useiden iteraatioiden aikana ja tulostamme ne jokaisen `report_freq`-koulutussyklin jälkeen.

Tämä koulutusprosessi kestää todennäköisesti melko kauan, joten rajoitamme iteraatioiden määrää.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Voit huomata (varsinkin, jos lisäät iteraatioiden määrää ja odotat tarpeeksi kauan), että BERT-luokittelu antaa meille varsin hyvän tarkkuuden! Tämä johtuu siitä, että BERT ymmärtää jo valmiiksi kielen rakenteen melko hyvin, ja meidän tarvitsee vain hienosäätää lopullista luokittelijaa. Kuitenkin, koska BERT on suuri malli, koko koulutusprosessi vie paljon aikaa ja vaatii huomattavaa laskentatehoa! (GPU, ja mieluiten useampi kuin yksi).

> **Huom:** Esimerkissämme olemme käyttäneet yhtä pienimmistä valmiiksi koulutetuista BERT-malleista. On olemassa suurempia malleja, jotka todennäköisesti tuottavat parempia tuloksia.


## Mallin suorituskyvyn arviointi

Nyt voimme arvioida mallimme suorituskykyä testidatalla. Arviointisilmukka on hyvin samanlainen kuin harjoitussilmukka, mutta meidän ei pidä unohtaa vaihtaa mallia arviointitilaan kutsumalla `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Tärkeimmät asiat

Tässä osiossa olemme nähneet, kuinka helppoa on ottaa valmiiksi koulutettu kielimalli **transformers**-kirjastosta ja mukauttaa se tekstiluokittelutehtäväämme. Samalla tavalla BERT-malleja voidaan käyttää entiteettien tunnistamiseen, kysymyksiin vastaamiseen ja muihin NLP-tehtäviin.

Transformer-mallit edustavat NLP:n nykyistä huipputasoa, ja useimmissa tapauksissa niiden pitäisi olla ensimmäinen ratkaisu, jota kokeilet, kun toteutat räätälöityjä NLP-ratkaisuja. Kuitenkin tämän moduulin käsittelemien toistuvien neuroverkkojen perusperiaatteiden ymmärtäminen on äärimmäisen tärkeää, jos haluat rakentaa edistyneitä neuroverkkomalleja.



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
